### Air Gas Ratio Optimal Policy

In [140]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torch.nn.init as init

from torch.utils.data import DataLoader, Dataset 

In [141]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 128)   
        self.fc3 = nn.Linear(128, 3)
        self.dropout_prob = 0.3
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(128)        

    def forward(self, x): 
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)        
        
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)                               
        
        x = self.fc3(x)
        return x
    
model_weights = torch.load('transition.pt')
model = Net()
model.load_state_dict(model_weights) 
model.eval()

Net(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (batch_norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [142]:
class Melter:
    
    def __init__(self, sample_data):
        self.actions = [0, 1, 2, 3, 4]
        self.sample_data = sample_data
        self.input_features = ['pre_heated_air_temp','inner_press_pv', 'inner_temp_pv', 'input_hard', 'action']
        self.count = 0
        self.action_list = []
        
        
    def reset(self):             
        self.state = np.array(self.sample_data.groupby('Batch').first().sample(1)[self.input_features])       
        self.start_temp = self.state[0][2]
        return self.state

    def random_action(self):
        sample = [0, 1, 2, 3, 4]
        return random.choice(sample)  
        
    def step(self, action):
        self.count += 1
        self.action_list.append(action)
        self.state[0][4] = action
        Yhat = model(torch.FloatTensor(X_scaler.transform(self.state)))
        Yhat = Yhat  + torch.FloatTensor(np.random.normal(0, 0.1, 3)) # add Gausian noise
        delta = Y_scaler.inverse_transform(Yhat.detach().numpy())
        delta = np.append(delta, [[0, 0]])
        next_state  = self.state + delta
        self.state = next_state        
        self.action_list.append(action)
        
        r1 = (self.state[0][2] - self.start_temp) / (self.count*3) # Fuel efficiency
        r2 = pd.Series(self.action_list).mean() # Fuel saving
        
        if (self.state[0][2]  > 960):   
            alpha = 0.8
            r = alpha*(r1 > 5).astype('int') + (1-alpha)*r2
            done = True                            
            self.count = 0            

        else:
            r = 0
            done = False
        
        info = ''       
        
        return next_state, r, done, info

In [143]:
import pandas as pd
import numpy as np
import os
import pickle
import random
import warnings
warnings.filterwarnings("ignore")

input_data = pd.read_pickle('input_data.pkl').sample(frac=1)

with open('x_scaler', 'rb') as file:
    X_scaler = pickle.load(file)
    
with open('y_scaler', 'rb') as file:
    Y_scaler = pickle.load(file) 

In [144]:
input_data.head()

,input_hard,pre_heated_air_temp,inner_press_pv,inner_temp_pv,airgas_ratio,action,time,preheat_delta,temp_delta,pressure_delta
Batch,,,,,,,,,,
S52576-1,13.020,178.314011,6.489959,860.135986,12.682740,2,2021-02-06 22:45:00,2.071884,18.497375,1.179018
S63047-1,15.845,207.033783,2.352199,835.379822,11.539531,0,2021-08-31 15:39:00,2.338440,21.186523,-0.294725
S57385-1,18.790,226.436050,6.122048,919.276856,12.705288,3,2021-05-14 04:33:00,4.459946,23.296814,0.506588
S50394-1,14.980,168.690185,5.064332,933.404907,11.538676,0,2020-12-27 10:21:00,4.159317,14.392853,0.226451
S50203-1,10.810,172.492310,7.850105,877.478882,11.581228,0,2020-12-22 11:21:00,5.287125,15.924744,0.230809


In [149]:
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0001
gamma         = 0.99
buffer_limit  = 15000
batch_size    = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n): # 버퍼에서 샘플링
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(5, 32)
        self.fc2 = nn.Linear(32, 32)   
        self.fc3 = nn.Linear(32, 5)

    def forward(self, x): # Q Value 리턴 (음수가 될 수 도 있음)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))    
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random() # 0 ~ 1 
        if coin < epsilon:
            return random.randint(0, 4)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size)
        q_out = q(s).reshape(batch_size , 5) # input size (32,10) return size (32,10)
        q_a = q_out.gather(1, a) # 취한 액션의 큐값만 골라냄 (32,1)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


q = Qnet()
q_target = Qnet()
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()

print_interval = 100
score = 0.0  
optimizer = optim.Adam(q.parameters(), lr=learning_rate) # q_target 은 업데이트 안 함!

melter_env = Melter(input_data)

for n_epi in range(2000):
    epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
    s = melter_env.reset()
    
    done = False

    while not done:
        a = q.sample_action(torch.from_numpy(s).float(), epsilon)  
        s_prime, r, done, info = melter_env.step(a)                         

        done_mask = 0.0 if done else 1.0
        memory.put((s, a, r, s_prime, done_mask))
        s = s_prime        
        score += r
        if done:        
            break


    if memory.size()>2000:
        train(q, q_target, memory, optimizer)

    if n_epi%print_interval==0 and n_epi!=0:
        
        q_target.load_state_dict(q.state_dict()) # 타겟 네트워크 업데이트 (20 번 에피소드 마다)
        print("n_episode :{}, score : {:.4f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval, memory.size(), epsilon*100))                
        
        if (score/print_interval) > 0.65:
            break

        score = 0.0

print('END')        

n_episode :100, score : 0.2538, n_buffer : 914, eps : 7.5%
n_episode :200, score : 0.2808, n_buffer : 1807, eps : 7.0%
n_episode :300, score : 0.3114, n_buffer : 2679, eps : 6.5%
n_episode :400, score : 0.3667, n_buffer : 3764, eps : 6.0%
n_episode :500, score : 0.3935, n_buffer : 4782, eps : 5.5%
n_episode :600, score : 0.4031, n_buffer : 5810, eps : 5.0%
n_episode :700, score : 0.4109, n_buffer : 6911, eps : 4.5%
n_episode :800, score : 0.4109, n_buffer : 8059, eps : 4.0%
n_episode :900, score : 0.4243, n_buffer : 9087, eps : 3.5%
n_episode :1000, score : 0.4436, n_buffer : 10193, eps : 3.0%
n_episode :1100, score : 0.4664, n_buffer : 11280, eps : 2.5%
n_episode :1200, score : 0.4816, n_buffer : 12443, eps : 2.0%
n_episode :1300, score : 0.4724, n_buffer : 13657, eps : 1.5%
n_episode :1400, score : 0.5088, n_buffer : 14849, eps : 1.0%
n_episode :1500, score : 0.5155, n_buffer : 15000, eps : 1.0%
n_episode :1600, score : 0.5316, n_buffer : 15000, eps : 1.0%
n_episode :1700, score : 0.

In [121]:
# torch.save(q_target.state_dict(), 'airgas_target.pt')
model_weights = torch.load('airgas_target.pt')
q_target = Qnet()
q_target.load_state_dict(model_weights)
q_target.eval()

Qnet(
  (fc1): Linear(in_features=5, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=5, bias=True)
)

In [150]:
import time

melter = Melter(input_data)

reward_list = []
action_list = []
for i_episode in range(500):
    
    s = melter.reset()
    done = False
    
    for t in range(20):                
        action = q_target(torch.Tensor(s)).argmax().item() 
        s_prime, reward, done, info = melter.step(action)
        s = s_prime
        action_list.append(action)
        
        if done:
            reward_list.append(reward)

            break

print(f'{pd.Series(reward_list).mean(): .2f}')       
print(pd.Series(action_list).value_counts().sort_index())

 0.64
1    1476
2      27
3    1280
4    3428
dtype: int64


In [151]:
melter = Melter(input_data)

reward_list = []
action_list = []

for i_episode in range(500):
    
    s = melter.reset()
    done = False
    
    for t in range(20):                
        action = random.choice([0, 1, 2, 3, 4])
        s_prime, reward, done, info = melter.step(action)
        s = s_prime
        action_list.append(action)    
        
        if done:
            reward_list.append(reward)
            break

print(f'{pd.Series(reward_list).mean(): .2f}')    
print(pd.Series(action_list).value_counts().sort_index())  

 0.43
0    952
1    962
2    953
3    927
4    993
dtype: int64


In [152]:
first_temp = input_data[input_data['inner_temp_pv']<=960].groupby('Batch')['inner_temp_pv'].first()
last_temp = input_data[input_data['inner_temp_pv']<=960].groupby('Batch')['inner_temp_pv'].last()
num_steps = input_data[input_data['inner_temp_pv']<=960].groupby('Batch').size()
action_mean = input_data[input_data['inner_temp_pv']<=960].groupby('Batch')['action'].mean()

first_index = first_temp[first_temp.between(740, 780)]
last_index =  last_temp[last_temp.between(930, 970)]
num_index = num_steps[num_steps.between(9, 18)]

# Batches for study
batches = list(set(list(last_index.index) + list(first_index.index) + list(num_index.index)))
last_temp2 = last_temp.loc[batches]
num_steps2 = num_steps.loc[batches]
action_mean2 = action_mean.loc[batches]


r1 = (last_temp2 - first_temp2)/(num_steps2*3) # add 3 multiple later as the data have a 3 minute time step.
r2 = action_mean2

alpha = 0.8
r = alpha*(r1>5) + (1-alpha)*r2

print(f'{r.mean():5.2f}')
print(input_data.loc[batches]['action'].value_counts().sort_index())

 0.39
0    2961
1     914
2     868
3    4029
4    1002
Name: action, dtype: int64
